In [22]:
%run "0. INSTALLS & GLOBALS.ipynb"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
MASTER_DF = pd.read_csv("outputs/Nobel_Laureates_Clean.csv")
MASTER_DF[:10]

,YEAR,LAUREATE,NAME_ORDER,URL,FIELD
0,1901,Jacobus Henricus van 't Hoff,1,/wiki/Jacobus_Henricus_van_%27t_Hoff,Chemistry
1,1901,Sully Prudhomme,1,/wiki/Sully_Prudhomme,Literature
2,1901,Emil von Behring,1,/wiki/Emil_von_Behring,Medicine
3,1901,Henry Dunant,1,/wiki/Henry_Dunant,Peace
4,1901,Frédéric Passy,2,/wiki/Fr%C3%A9d%C3%A9ric_Passy,Peace
5,1901,Wilhelm Röntgen,1,/wiki/Wilhelm_R%C3%B6ntgen,Physics
6,1902,Emil Fischer,1,/wiki/Emil_Fischer,Chemistry
7,1902,Theodor Mommsen,1,/wiki/Theodor_Mommsen,Literature
8,1902,Ronald Ross,1,/wiki/Ronald_Ross,Medicine
9,1902,Élie Ducommun,1,/wiki/%C3%89lie_Ducommun,Peace


# Get Biography
DOB, DOD (Date of Death) (I don't know what else to call it.), Place of Birth, Place of Death, Date of Nobel Award

In [ ]:
def get_biography(page):
    information_of_interest = ["born", "died", "nationality", 'alma mater', "award"]
    biography_table = page.xpath("//*[contains(@class, 'vcard')]//tr")

    #if sibling element have [word], scrape content
    #born = biography_table[0].xpath(".//tr/th[contains(., 'born')]/following-sibling::*")

    for item in biography_table: #iterating through rows of biography table
        
        print(f"{item.tag}: {item.text_content()}")
        
    return biography_table

# Driver Code

In [75]:
for row in MASTER_DF[:10].itertuples():
    url = WIKIPEDIA_URL + row[4]
    page = GET_ETREE(DRIVER, url, parser="html")

    #debug
    print(f"Scraping: {url}")

    bio_data = get_biography(page)

Scraping: https://en.wikipedia.org/wiki/Jacobus_Henricus_van_%27t_Hoff
tr: Jacobus Henricus van 't Hoff Jr.
tr: Van 't Hoff in 1904
tr: Born(1852-08-30)30 August 1852Rotterdam, Netherlands
tr: Died1 March 1911(1911-03-01) (aged 58)Steglitz, Berlin, German Empire
tr: Alma mater.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}
Delft University of Technology
University of Leiden
University of Bonn
University of Paris
University of Utrecht

tr: Known for
One of the founders of stereochemistry and physical chemistry
Modern concept of chemical affinity
Predicting the structure of tetrahedral carbon atom (asymmetric carbon)
Predicting the existence of chiral allenes
Stereoisomerism
Chemical thermodynamics
Chemical kinetics
Chemical equilibrium
Osmotic pressure
Van 't Hoff equation
Van 't Hoff factor
Le Bel–Van 't Hoff rule

tr: Awards
Davy